In [1]:
#Import statements
import os
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [4]:
#How to load all the in once?
#Load in the PDFS
#loaders = [PyPDFLoader("C:/Users/elias/Downloads/WEF_The_Global_Cooperation_Barometer_2024.pdf")]

#loader = UnstructuredPDFLoader(file_path="C:/Users/elias/Downloads/WEF_The_Global_Cooperation_Barometer_2024.pdf")
'''
For one PDF this method works!:

loader = PyPDFLoader("C:/Users/elias/Downloads/HPE ProLiant DX360 Gen10 Plus server-a50002577enw.pdf")
data = loader.load()
docs = []

for file in loaders:
    docs.extend(file.load())
    
data[2].page_content
'''
#Test to extends to multiple pdfs in once:
pdf_folder_path = "/Users/keithatienza/Desktop/Academics/Emergent Consulting [HPE]/HPE LLM v2/HPE Files"
documents = []
for file in os.listdir(pdf_folder_path):
    if file.endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder_path, file)
        loader = PyPDFLoader(pdf_path)
        documents.extend(loader.load())

In [6]:
documents

[Document(metadata={'source': '/Users/keithatienza/Desktop/Academics/Emergent Consulting [HPE]/HPE LLM v2/HPE Files/HPE ProLiant ML350 Gen11-a50004308enw.pdf', 'page': 0}, page_content="QuickSpecs  HPE ProLiant ML350 Gen11  \nOverview  \n \n \n  \nPage 1 \n \nHPE ProLiant ML350 Gen11   \nProvide most powerful and storage flexibility 2P tower server with rackable chassis design for various environments, and deli vers \nexceptional compute performance, security, reliability, and expandability. Design to fulfill with wide range workloads for small \noffices, remote and branch offices of large enterprises, growing SMBs and data centers. HPE ProLiant ML350 Gen11 server is an  \nexcellent choice to a ccelerate your growing business.  \n \n \n \nFront View – LFF chassis with optional Gen11 4LFF HDD Cage Kits shown ( Tower mode)  \n1. DisplayPort 1.1a   8. Optical drive bay  \n2. USB 3.2 Gen1 port   9. Media bay filler panel  \n3. iLO service port   10. Serial number/iLO information pull tab  

In [8]:
#Split the pdfs
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = text_splitter.split_documents(documents)

In [10]:
chunks[100]

Document(metadata={'source': '/Users/keithatienza/Desktop/Academics/Emergent Consulting [HPE]/HPE LLM v2/HPE Files/HPE ProLiant ML350 Gen11-a50004308enw.pdf', 'page': 35}, page_content='− 4800 MT/s maximum memory speed.  \n− 512GB SGX Enclave.  \n− Only supported in single socket configuration.   \nIntel Xeon -Platinum 8558P 2.7GHz 48-core 350W Processor for HPE  P67108 -B21 \nNotes:  \n− Requires Redundant Fan Kit (P47219 -B21) and Second CPU Fan Kit (P47902 -B21).  \n− 512GB SGX Enclave.   \nIntel Xeon -Gold Processors   \nIntel Xeon -Gold 6554S 2.2GHz 36 -core 270W Processor for HPE  P67110 -B21 \nNotes:  5200 MT/s maximum memory speed.   \nIntel Xeon -Gold 6530 2.1GHz 32 -core 270W Processor for HPE  P67095 -B21 \nNotes:  4800 MT/s maximum memory speed.   \nIntel Xeon -Gold 6548Y+ 2.5GHz 32 -core 250W Processor for HPE  P67082 -B21 \nNotes:  5200 MT/s maximum memory speed.   \nIntel Xeon -Gold 6548N 2.8GHz 32 -core 250W Processor for HPE  P67105 -B21 \nNotes:  5200 MT/s maximum mem

In [12]:
#Load in a method to do the embeddings
!ollama pull nomic-embed-text

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB                         
pulling c71d239df917... 100% ▕████████████████▏  11 KB                         
pulling ce4a164fc046... 100% ▕████████████████▏   17 B                         
pulling 31df23ea7daa... 100% ▕████████████████▏  420 B                         
verifying sha256 digest 
writing manifest 
success 


In [14]:
# Create new vector database with all the documents that are loaded
vector_db = Chroma.from_documents(
    documents=chunks, 
    embedding=OllamaEmbeddings(model="nomic-embed-text",show_progress=True),
    collection_name="local-rag", persist_directory="/Users/keithatienza/Desktop/Academics/Emergent Consulting [HPE]/HPE LLM v2/Database"
)

/var/folders/1g/drrbdl5d5tz58v9sn33gqfb40000gn/T/ipykernel_1584/2791099157.py:4: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding=OllamaEmbeddings(model="nomic-embed-text",show_progress=True),
OllamaEmbeddings: 100%|███████████████████████| 187/187 [00:09<00:00, 20.36it/s]


In [ ]:
#Function to add something to existing Chroma DB so not entire DB has to be created.

In [30]:
vector_db.persist()

NameError: name 'vector_db' is not defined

In [32]:
#Retrieve DB
vector_db = Chroma(persist_directory=persist_directory, embedding_function=OllamaEmbeddings(model="nomic-embed-text",show_progress=True))

NameError: name 'persist_directory' is not defined

In [14]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [15]:
#Load in LLM
!ollama pull mistral

pulling manifest â ‹ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¸ pulling manifest â ´ pulling manifest â ´ pulling manifest â § pulling manifest â § pulling manifest â � pulling manifest 
pulling ff82381e2bea... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 4.1 GB                         
pulling 43070e2d4e53... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  11 KB                         
pulling 491dfa501e59... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  801 B                         
pulling ed11eda7790d... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�   30 B                         
pulling 42347cd80dc8... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  485 B                         
verifying sha256 digest 
writing manifest 
success 


In [16]:
local_model = "mistral"
llm = ChatOllama(model=local_model)

C:\Users\elias\AppData\Local\Temp\ipykernel_16772\3508398677.py:2: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model=local_model)


In [17]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [18]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [19]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [20]:
chain.invoke(input(""))

How many GPUs can fit in a Proliant DL380a Gen11 server?


OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.15s/it]


' The provided specifications do not mention GPU compatibility for the HPE ProLiant DL360 Gen11 server model mentioned in your text. To find out about GPU compatibility, you should check the technical specifications and documentation for the specific model of HPE ProLiant DL380a Gen11 server that you are interested in. Different models may have varying GPU support.'

In [21]:
chain.invoke(input(""))

About which products do you have information


OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.14s/it]


" The provided information pertains to three different pre-configured models of HPE ProLiant servers, namely the DL345 Gen11, DL380 Gen11, and DL360 Gen11. Each model is available in various regions as indicated by the country code key (Worldwide, Japan, Europe, the Middle East and Africa, China, and India). Additionally, it seems that these products are part of HPE's Smart Choice purchase program, which offers quick shipping, pre-configured products, and aggressively priced HPE Tech Care Services."

In [22]:
chain.invoke(input(""))

Is the RTX 4000 GPU supported in any of HPE’s server models?


OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.14s/it]


" Based on the provided information, the specific model of GPU (RTX 4000) is not mentioned in the QuickSpecs for either the HPE ProLiant DL380a Gen11 or the HPE ProLiant DL380 Gen11. However, it is stated that NVIDIA GPUs are supported, and some models (such as the H100) are explicitly listed. To find out if RTX 4000 GPU is supported in any of HPE's server models, further research would be required by checking more detailed specifications or contacting a local sales representative for additional information."